## Example of Identifying Reference Object in Scene

Source: https://dev.to/erol/object-detection-with-color-knl

In [63]:
import cv2
import numpy as np
from collections import deque
import math
import os.path

In [ ]:
buffer_size = 16
pts = deque(maxlen=buffer_size)

In [3]:
# Using HSV: Hue, Saturation and Value for representing color in images to then detect
# Define upper and lower thresholds for the reference object in hsv
# ref_lower = (0, 0, 150)
# ref_upper = (255, 20, 255)

# ref_lower = (0, 0, 0)
# ref_upper = (255, 70, 30)

ref_lower = (100, 100, 100)
ref_upper = (140, 255, 200)

In [4]:
# Read image
image = cv2.imread("C:/Users/jzapu/Image_Scaling/mspaint_test.jpg")
# imaget = cv2.imread("C:/Users/jzapu/Image_Scaling/img2.jpg")

In [5]:
# We will be applying gaussian smoothing, then HSV conversion, then opening (Erosian -> Dilation) to preprocess image for targeting
# Gaussian Operation
blur_image = cv2.GaussianBlur(image, (11, 11), 0)
# RGB -> HSV
hsv = cv2.cvtColor(blur_image, cv2.COLOR_BGR2HSV)
# Morphology: Opening Image
# Define Mask for Operation
mask = cv2.inRange(hsv, ref_lower, ref_upper)
mask = cv2.erode(mask, None, iterations=2)
mask = cv2.dilate(mask, None, iterations=2)

In [6]:
# Now noise is gone: find edges of reference object (all edges)
contours,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
center = None

In [7]:
# Find coordinates in image to draw the target shape outline
if (len(contours) > 0):
    target = max(contours, key=cv2.contourArea)
    # (x,y), radius = cv2.minEnclosingCircle(target)
    rect = cv2.minAreaRect(target)
    # center = (int(x),int(y))
    # radius = int(radius)
    ((x,y), (width, height), rotation) = rect
    s = f"x {np.round(x)}, y: {np.round(y)}, width: {np.round(width)}, height: {np.round(height)}, rotation: {np.round(rotation)}"
    box = cv2.boxPoints(rect)
    box = np.int64(box)
    M = cv2.moments(target)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    # out = cv2.circle(image, center, radius, (0, 255, 0), 4)
    cv2.circle(image, center, 5, (255, 0, 255), -1)
    cv2.drawContours(image, [box], 0, (255, 0, 0), 4)
    fin =  cv2.putText(image, s, (25, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (200, 0, 50), 2)
    cv2.imwrite('testout.jpg', fin)
    cv2.imshow("TEST", fin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(center, rect)
else:
    print("Error with thresholds")

(1810, 2023) ((1810.9439697265625, 2020.49560546875), (445.7012939453125, 324.5749816894531), 89.41537475585938)


In [ ]:
# For identifying multiple windows in scene - will not use for P2
num_windows = 0
rect_arr = []
box_arr = []
for i in range(len(contours)):
    target = max(contours, key=cv2.contourArea)
    num_windows = num_windows + 1
    rect = cv2.minAreaRect(contours[i])
    rect_arr.append(rect)
    ((x,y), (width, height), rotation) = rect
    box = cv2.boxPoints(rect)
    box = np.int64(box)
    box_arr.append(box)
    M = cv2.moments(target)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    #cv2.circle(image, center, 5, (255, 0, 255), -1)
    cv2.drawContours(image, [box], 0, (255, 0, 0), 4)


In [8]:
# If needed, crop image to bottom of wall
im = cv2.imread('./p2_test3.jpeg', cv2.IMREAD_GRAYSCALE)
cropped_image = im[0:box[3][1], 0:4032]
cv2.imwrite("cropped.jpg", cropped_image)

True

In [ ]:
# Checking where opencv decided to order the box edges 
for i in box:
    cv2.circle(image,(i[0],i[1]), 3, (0,255,0), -1)
    cv2.imshow("TEST", fin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Perspective Transform
srcpts = np.float32([box[0], box[1], box[2], box[3]])


True

In [ ]:
# Gets ordered cw from upper left (index 1 is row not 0 - wtf)
print(image.shape)
print(box[0][1])
print(box[1][1])
print(box[0][0])
print(box[1][0])
print(box[1])
print(box[0])
print(box[2])
print(box[3])
print(box[0][0])
print(box[3][0])
print(box[2][0])
print(box)
print(box[3][1])

In [9]:
# Getting pixel -> feet conversion
# Define Wight and Height of the reference in pixels
refWidth = 2.0
refHeight = 3.0
# Ratio: Pixels / Foot
widthRatio = refWidth / width
heightRatio = refHeight / height
# Dims of image
rows, cols, ch = image.shape
sideHeight = heightRatio * rows
sideWidth = widthRatio * cols
print(sideWidth, sideHeight)

18.092835065875875 27.95039824936325


In [10]:
# Write image for white - black image
image[image != 0] = 255 # change everything to white where pixel is not black
cv2.imwrite('bw_image.jpg', image)

True

In [11]:
# Car width in feet
cart_width = 0.427
# Car width in pixels 
block_width = math.floor(cart_width / widthRatio)
# Car height in feet
cart_height = 0.558
# Car height in pixels 
block_height = math.floor(cart_height / heightRatio)
print(block_height, block_width)

60 95


In [12]:
# Get length and width of image 
matrix_rows = math.floor(rows / block_height)
matrix_cols = math.floor(cols / block_width)
matrix_out = [[0 for x in range(matrix_cols)] for y in range(matrix_rows)]

In [ ]:

# Need greyscale for comparing pixel values
im_gray = im
cv2.imwrite('bw_image.jpg', im_gray)

True

In [ ]:

# Loop over image and fill output matrix for path plan
clean_threshold = 80
start_pixel_row = 0
clean_blocks = 0
for i in range(matrix_rows):
    # Set starting pixel starting points
    start_pixel_col = 0
    for j in range(matrix_cols):
        #print("j = {}, i = {}".format(i, j))
        # Loop over all pixels in block 
        #   if # black pixels greater than threshold then no go - o.w. yes clean -> 255
        zero_count = 0
        for u in range(start_pixel_row, start_pixel_row + block_height):
            for v in range(start_pixel_col, start_pixel_col + block_width):
                #print("u = {}, v = {}".format(u, v))
                if im[u, v] == 0:
                    zero_count = zero_count + 1
        if zero_count < clean_threshold:
            matrix_out[i][j] = 255;
            clean_blocks = clean_blocks + 1
        # Update starting pixel values
        # Row value same
        # Col value inc by block width
        start_pixel_col = start_pixel_col + block_width
    start_pixel_row = start_pixel_row + block_height

In [85]:
# Convert to image 
final_blocks = np.asarray(matrix_out)
cv2.imwrite('scale_out.jpg', final_blocks)

True

In [86]:
# Pad matrix with 0s 
blocks = np.pad(final_blocks, 1)

In [77]:
print(blocks.shape)

(52, 44)


In [ ]:
with open('block_file.txt', 'w') as f:
    for item in final_blocks:
        f.write("%s\n" % item)

In [83]:
# Create File for video ouput 
save_path = 'C:/Users/jzapu/Image_Scaling/OutputArr'

# Getting Instruction Set 'L,R,U,D' for Robot

In [87]:
# Need to interpolate size of image to see it properly 
scale_percent = 60
width = int(blocks.shape[1] * scale_percent)
height = int(blocks.shape[0] * scale_percent)
dim = (width, height)
# Instructions 
instr_priorities = ['D', 'L', 'R', 'U', 'B']
instr_set = []
# D = Down; L = Left; R = Right; U = Up, B = Back (Reverse)
# Need a visited list for reverse function 
visited = []
# Need to track if all blocks have been cleaned
cleaned = 1
# Number of steps 
steps = 0
index = 0
# Start Position is lower right corner 
start_pos = (50, 42)
blocks[50, 42] = 0
current_pos = start_pos
row = 50
col = 42
# Push starting block to visited stack
visited.append(start_pos)
# Loop over matrix
while (cleaned < clean_blocks):
    # Check Down
    if (blocks[row+1][col] == 255):
        # Append Instruction
        instr_set.append('D')
        # Update cleaned block
        blocks[row+1][col] = 0
        # Count Steps Taken, current position & update row, col
        steps = steps + 1
        row = row + 1
        index = index + 1
        cleaned = cleaned + 1
        current_pos = (row, col)
        # Push block to visited
        visited.append(current_pos)
        # Write image for video representation
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks)
        i = cv2.imread(complete_name, cv2.IMREAD_UNCHANGED)
        blocks_resized = cv2.resize(i, dim, interpolation=cv2.INTER_CUBIC)
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks_resized)
        # Continue loop
        continue
    # Check Left
    elif (blocks[row][col-1] == 255):
        # Append Instruction
        instr_set.append('L')
        # Update cleaned block
        blocks[row][col-1] = 0
        # Count Steps Taken, current position & update row, col
        steps = steps + 1
        index = index + 1
        col = col - 1
        cleaned = cleaned + 1
        current_pos = (row, col)
        # Push block to visited
        visited.append(current_pos)
        # Write image for video representation
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks)
        i = cv2.imread(complete_name, cv2.IMREAD_UNCHANGED)
        blocks_resized = cv2.resize(i, dim, interpolation=cv2.INTER_CUBIC)
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks_resized)
        # Continue loop
        continue
    # Check Right 
    elif (blocks[row][col+1] == 255):
        # Append Instruction
        instr_set.append('R')
        # Update cleaned block
        blocks[row][col+1] = 0
        # Count Steps Taken, current position & update row, col
        steps = steps + 1
        index = index + 1
        col = col + 1
        cleaned = cleaned + 1
        current_pos = (row, col)
        # Push block to visited
        visited.append(current_pos)
        # Write image for video representation
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks)
        i = cv2.imread(complete_name, cv2.IMREAD_UNCHANGED)
        blocks_resized = cv2.resize(i, dim, interpolation=cv2.INTER_CUBIC)
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks_resized)
        # Continue loop
        continue
    # Check Up
    elif (blocks[row-1][col] == 255):
        # Append Instruction
        instr_set.append('U')
        # Update cleaned block
        blocks[row-1][col] = 0
        # Count Steps Taken, current position & update row, col
        steps = steps + 1
        index = index + 1
        row = row - 1
        cleaned = cleaned + 1
        current_pos = (row, col)
        # Push block to visited
        visited.append(current_pos)
        # Write image for video representation
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks)
        i = cv2.imread(complete_name, cv2.IMREAD_UNCHANGED)
        blocks_resized = cv2.resize(i, dim, interpolation=cv2.INTER_CUBIC)
        file_name = 'video{}.jpg'.format(cleaned)
        complete_name = os.path.join(save_path, file_name)
        cv2.imwrite(complete_name, blocks_resized)
        # Continue loop
        continue
    # If not Reverse 
    else:
        # Append Instruction
        instr_set.append('B')
        # Count Steps Taken, current position & update row, col
        # Get former position
        current_pos = visited[index-1]
        # Pop Current Block
        visited.pop()
        # Update Steps 
        steps = steps + 1
        index = index - 1
        row = current_pos[0]
        col = current_pos[1]
        # Continue loop
        continue


In [88]:
print(steps)
print(row, col)
print(cleaned)

1694
1 41
1630


In [89]:
with open('InstructionSet.txt', 'w') as f:
    for item in instr_set:
        f.write("%s" % item)

In [90]:
# Create Video Represenation of Cleaning 
video_name = 'video.avi'
images = [img for img in os.listdir(save_path) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(save_path, images[0]))
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, 0, 1, (width, height))
for i in images:
    video.write(cv2.imread(os.path.join(save_path, i)))

cv2.destroyAllWindows()
video.release()

### Below are for getting all dimensions - not needed here

In [ ]:
# Get Total length and width of the side
#############################################
# Dims of image
rows, cols, ch = image.shape
sideHeight = heightRatio * cols
sideWidth = widthRatio * rows
print(sideWidth, sideHeight)

# Draw Lines
image = cv2.line(image, (0, 0), (0, 500), (0, 255, 255), 2)
image = cv2.line(image, (0, 0), (500, 0), (0, 255, 255), 2)

9.072000000000001 25.401600000000002


In [ ]:
# Get end-of-image to first / last window distance
# First Window:
print(num_windows)
print(box_arr[0])
print(box_arr[3])
left_padding = box_arr[0][0][0] - 0
# Last Window
right_padding = cols - box_arr[2][1][0] 
# NOTE: Looks like cv2 going CCW with the box points 
print(left_padding, right_padding)

# Draw Lines
left_start_point = (0, box_arr[0][0][1])
left_end_point = (box_arr[0][0][0], box_arr[0][0][1])
right_start_point = (cols, box_arr[2][1][1])
right_end_point = (box_arr[2][1][0], box_arr[2][1][1])
image = cv2.line(image, left_start_point, left_end_point, (0, 255, 255), 2)
image = cv2.line(image, right_start_point, right_end_point, (0, 255, 255), 2)

8
[[4014 1633]
 [4019 1633]
 [4019 1637]
 [4014 1637]]
[[3707 1205]
 [3711 1205]
 [3711 1210]
 [3707 1210]]
4014 8


In [ ]:
# Get Top - bottom distance (top / bottom of image - to - window blackout)
bottom_padding = cols - box_arr[0][2][1]
top_padding = box_arr[0][2][1] - 0

# Draw Lines

image = cv2.line(image, (box_arr[3][0][0], 0), (box_arr[3][0][0], box_arr[3][0][1]), (0, 255, 255), 2)
image = cv2.line(image, (box_arr[3][0][0], box_arr[3][3][1]), (box_arr[3][0][0], cols), (0, 255, 255), 2)

In [ ]:
# Get Distance between windows 
# MAP
# arr[4] -> 1
# arr[6] -> 2
# arr[3] -> 3

d1 = box_arr[6][0][0] - box_arr[4][1][0]
d2 = box_arr[3][0][0] - box_arr[6][1][0]
image = cv2.line(image, (box_arr[4][1][0], box_arr[4][1][1]), (box_arr[6][0][0], box_arr[6][0][1]), (0, 255, 255), 2)
image = cv2.line(image, (box_arr[6][1][0], box_arr[6][1][1]), (box_arr[3][0][0], box_arr[3][0][1]), (0, 255, 255), 2)

In [ ]:
# SHow and Save
cv2.imshow("TEST", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('LinesDrawn.jpg', image)

In [ ]:
cv2.imshow('Windows Space Determined', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('windowOut.jpg', image)

True

In [ ]:
print(box_arr[0])
print(box_arr[1])
print(box_arr[2])
print(box_arr[3])
print(box_arr[4])
print(box_arr[5])
print(box_arr[6])

[[ 43 443]
 [ 47 443]
 [ 47 449]
 [ 43 449]]
[[ 43 383]
 [ 47 383]
 [ 47 388]
 [ 43 388]]
[[448 270]
 [453 270]
 [453 274]
 [448 274]]
[[363 267]
 [449 265]
 [454 456]
 [368 458]]
[[ 47 261]
 [132 261]
 [132 458]
 [ 47 458]]
[[439 260]
 [443 260]
 [443 264]
 [439 264]]
[[207 255]
 [295 255]
 [295 452]
 [207 452]]


In [ ]:
# Perspective transform
# NOTE: MAY WANT TO ADD A ROTATION IF ON A SLANT LIKE IN THIS IMAGE

# Dims of image
rows, cols, ch = image.shape

print (rows, cols)


3770 2501


In [ ]:
# Get feet / pixel value for reference - use for getting full image 
pwidth = rect[1][0]
pheight = rect[1][1]
# Calculate the Feet / Pixel
wratio = refWidth / pwidth
hratio = refHeight / pheight

print(wratio, hratio)

0.0030110388831515737 0.0062926639901208555


In [ ]:
# Output length and width of wall
sideHeight = hratio * cols
sideWidth = wratio * rows
print(sideWidth, sideHeight)

11.351616589481432 15.73795263929226


In [ ]:
print(box)

[[1376 2434]
 [2011 2413]
 [2045 3408]
 [1409 3430]]


In [ ]:
# Perspective Transform 
# 2DBox starts from bottom right corner and goes clockwise
ppts = np.float32(box)
pts = np.float32([[refWidth, 0], [0, 0], [0, refHeight], [refWidth, refHeight]])
M = cv2.getPerspectiveTransform(ppts, pts)
print(M)

[[-4.72887730e-03  1.61589777e-04  9.11985612e+00]
 [ 1.33161157e-04  4.02653976e-03 -9.98382753e+00]
 [-1.47211344e-06  1.68547735e-06  1.00000000e+00]]


In [ ]:
out = cv2.warpPerspective(image, M, (refWidth, refHeight))

error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'warpPerspective'
> Overload resolution failed:
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type
>  - Can't parse 'dsize'. Sequence item with index 0 has a wrong type


In [ ]:
cv2.imshow("HERE", out)
cv2.waitKey(0)

-1